In [89]:
import pandas as pd
import numpy as np
import xarray as xr
import dask.array as da
import dask
import dask_gateway
import os
import rioxarray as rioxr
from random import sample

from shapely.geometry import Point 
import geopandas as gpd

In [ ]:
# # initialize DASK cluster
# cluster = dask_gateway.GatewayCluster()
# cluster.scale(30)

# client = cluster.get_client()
# client

In [92]:
# Load raster
fp = os.path.join(os.getcwd(),
                  'temp', 
                  'S_clip_preds_2020',
                  'S_filter_clip_preds_ca_m_3411933_ne_11_060_20200522.tif')

#raster = rioxr.open_rasterio(fp, chunks={"x": 30000, "y":20000})       
raster = rioxr.open_rasterio(fp).squeeze()
raster

<xarray.DataArray (y: 1950, x: 5199)>
[10138050 values with dtype=uint8]
Coordinates:
    band         int64 1
  * x            (x) float64 2.299e+05 2.299e+05 2.299e+05 ... 2.33e+05 2.33e+05
  * y            (y) float64 3.815e+06 3.815e+06 ... 3.814e+06 3.814e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [29]:
# count number of pixels in each category

# 0 = noniceplant
# 1 = iceplant
# 2 = ground
# 3 = water

#cats = np.unique(raster)
cats = [0,1,2,3]
n_pix = []
for cat in cats:
    n_pix.append(len(np.where(raster == cat)[0]))
n_pix   

[4401242, 236072, 1924182, 389701]

In [ ]:
# Recreation of sampling design by SEPAL

# ---------------------------------------------
# --------------- PARAMETERS ------------------
# standard error for all the points
std_error = 0.01

# users accuracies (estimates)
u = [0.7, 0.7, 0.9, 0.9]
# ---------------------------------------------

# fraction of pixels with a given class in total pixels
total_pix = sum(n_pix)
pix_prop = [x/total_pix for x in n_pix]

# standard deviation of u
stdv = [ np.sqrt(x*(1-x)) for x in u]

X = [ x*y for x,y in zip(pix_prop, stdv)]

sample_size = (sum(X)/std_error)**2
sample_size

# distributing sample size among classes
#[...]

In [83]:
# arrays with indices where condition is True
coords = np.where(raster==0)

# sample random (y,x) pairs from list
indices = sample(list(zip(coords[0],coords[1])), 5)

# unzip into y and x coordinates
y, x = list(zip(*indices))

# x and y coordinates in raster CRS corresponding to sampled points
x_coord = raster.x[np.asarray(x)]
y_coord = raster.y[np.asarray(y)]

In [84]:
# make points 
x_coord

<xarray.DataArray 'x' (x: 5)>
array([231317.7, 231557.7, 230828.1, 231498.3, 230928.3])
Coordinates:
    band         int64 1
  * x            (x) float64 2.313e+05 2.316e+05 2.308e+05 2.315e+05 2.309e+05
    spatial_ref  int64 0

In [87]:
points = [ Point(i,j) for i,j in zip(x_coord, y_coord)]

In [96]:
samples = gpd.GeoDataFrame({'geometry': points}, crs=raster.rio.crs)

In [98]:
samples.to_file('samples.shp')